## 🧠 **CSV-Based Retrieval-Augmented Generation (RAG)** | **RAG100X**

This notebook presents a lightweight Retrieval-Augmented Generation (RAG) pipeline built to extract insights from structured CSV files. It processes tabular customer data, breaks it into retrievable text segments, and leverages vector embeddings to enable natural language question-answering over structured records.



✅ **Key Capabilities**  
*This notebook demonstrates a functional RAG pipeline tailored for CSV files:*

- *Loads and parses structured CSV documents using LangChain*  
- *Splits data into semantically useful chunks*  
- *Converts each chunk into dense vector embeddings via OpenAI*  
- *Stores the vectors in a FAISS index for fast retrieval*  
- *Retrieves and ranks relevant records based on user queries*  
- *Generates concise answers grounded in the CSV content*


> 🛠️ **Note:** The full pipeline is implemented directly inside the notebook, with no reliance on external helper modules—ensuring transparency and reproducibility for future reference.


### 📦 Installing Core RAG Libraries

These installations bring in all the necessary tools to implement a basic RAG pipeline:

- **`langchain`, `langchain-community`, `langchain-openai`**  
  Provides modular tools to load, embed, and retrieve documents with OpenAI models.

- **`faiss-cpu`**  
  An efficient similarity search library used to store and query vector embeddings.

- **`python-dotenv`**  
  Loads environment variables like your OpenAI API key from a `.env` file.

- **`pandas`**  
  Essential for reading and inspecting structured CSV data.

> Together, these packages let us go from raw CSV → document chunks → vector index → question answering.


In [3]:
!pip install -q langchain langchain-community langchain-openai faiss-cpu python-dotenv pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


### 🔐 Setting Up Your OpenAI API Key

To use OpenAI’s embedding models (like `text-embedding-3-small`), you’ll need an API key.

Run the cell below and **enter your API key securely** when prompted. It won’t be stored or exposed in this notebook.

⚠️ Quick Heads-Up

🧱 If you’re on a free OpenAI account without billing info:

❌ You won’t be able to use the embeddings API.

🤷‍♂️ You’ll need to upgrade or connect a payment method.

### 🧪 Workarounds (If You Hit a Wall)

No OpenAI access? No problem — here are plug-and-play alternatives:

- ✅ **HuggingFace Embeddings (Free & Local):**
  ```python
  from langchain.embeddings import HuggingFaceEmbeddings
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
  ```

In [ ]:
import os
from getpass import getpass

# Force prompt every time
os.environ["OPENAI_API_KEY"] = getpass("🔐 Enter your OpenAI API key: ")

### 📊 Loading the CSV Data

We begin by loading the customer dataset from a local CSV file using `pandas`.  
This gives us a quick preview of the data we'll be using for retrieval.


In [2]:
import pandas as pd

csv_path = "customers-100.csv"
df = pd.read_csv(csv_path)
df.head()

,Index,Customer Id,First Name,Last Name,Company,City,Country,Phone 1,Phone 2,Email,Subscription Date,Website
0,1,DD37Cf93aecA6Dc,Sheryl,Baxter,Rasmussen Group,East Leonard,Chile,229.077.5154,397.884.0519x718,zunigavanessa@smith.info,2020-08-24,http://www.stephenson.com/
1,2,1Ef7b82A4CAAD10,Preston,Lozano,Vega-Gentry,East Jimmychester,Djibouti,5153435776,686-620-1820x944,vmata@colon.com,2021-04-23,http://www.hobbs.com/
2,3,6F94879bDAfE5a6,Roy,Berry,Murillo-Perry,Isabelborough,Antigua and Barbuda,+1-539-402-0259,(496)978-3969x58947,beckycarr@hogan.com,2020-03-25,http://www.lawrence.com/
3,4,5Cef8BFA16c5e3c,Linda,Olsen,"Dominguez, Mcmillan and Donovan",Bensonview,Dominican Republic,001-808-617-6467x12895,+1-813-324-8756,stanleyblackwell@benson.org,2020-06-02,http://www.good-lyons.com/
4,5,053d585Ab6b3159,Joanna,Bender,"Martin, Lang and Andrade",West Priscilla,Slovakia (Slovak Republic),001-234-203-0635x76146,001-199-446-3860x3486,colinalvarado@miles.net,2021-04-17,https://goodwin-ingram.com/


### 📄 Converting CSV Rows into LangChain Documents

We use **`CSVLoader`** from `langchain_community` to convert each row in the CSV into a structured `Document` object.

- **`CSVLoader`**  
  Reads the CSV file and treats each row as a separate document — perfect for customer data, FAQs, or tabular records.

- **`.load_and_split()`**  
  Loads and automatically splits the CSV rows into individual documents for downstream embedding and retrieval. Even if we don’t manually define a splitter, the .load_and_split() method does internally chunk the CSV rows into document-like text chunks using the **RecursiveCharacterTextSplitter**.

>LangChain internally uses **RecursiveCharacterTextSplitter** by default.  

> This step transforms raw tabular data into a searchable format that LangChain understands.


In [4]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path=csv_path)
documents = loader.load_and_split()
documents

[Document(metadata={'source': 'customers-100.csv', 'row': 0}, page_content='Index: 1\nCustomer Id: DD37Cf93aecA6Dc\nFirst Name: Sheryl\nLast Name: Baxter\nCompany: Rasmussen Group\nCity: East Leonard\nCountry: Chile\nPhone 1: 229.077.5154\nPhone 2: 397.884.0519x718\nEmail: zunigavanessa@smith.info\nSubscription Date: 2020-08-24\nWebsite: http://www.stephenson.com/'),
 Document(metadata={'source': 'customers-100.csv', 'row': 1}, page_content='Index: 2\nCustomer Id: 1Ef7b82A4CAAD10\nFirst Name: Preston\nLast Name: Lozano\nCompany: Vega-Gentry\nCity: East Jimmychester\nCountry: Djibouti\nPhone 1: 5153435776\nPhone 2: 686-620-1820x944\nEmail: vmata@colon.com\nSubscription Date: 2021-04-23\nWebsite: http://www.hobbs.com/'),
 Document(metadata={'source': 'customers-100.csv', 'row': 2}, page_content='Index: 3\nCustomer Id: 6F94879bDAfE5a6\nFirst Name: Roy\nLast Name: Berry\nCompany: Murillo-Perry\nCity: Isabelborough\nCountry: Antigua and Barbuda\nPhone 1: +1-539-402-0259\nPhone 2: (496)978-3

### 🧠 Creating Embeddings & Building the Vector Store (FAISS)

This block does two things:

- **`OpenAIEmbeddings`**  
  Converts text chunks (documents) into vectors using OpenAI’s `"text-embedding-3-small"` model — fast, cost-effective, and works well for CSV/text-based tasks.

- **`FAISS.from_documents()`**  
  Takes the embedded documents and stores them in a FAISS index. This one-liner is ideal for rapid development.

> Great for getting started quickly — but if you want more control over how the index is built or stored, there’s another way.

---
### 🔧 Want More Control Over FAISS?

You can also build the vector store manually for finer control over indexing behavior or storage:

```python
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 1. Initialize the embedding model
embedding_model = OpenAIEmbeddings()

# 2. Get embedding dimension by embedding a dummy string
dim = len(embedding_model.embed_query("test"))

# 3. Create a raw FAISS index (FlatL2 = standard similarity search)
index = faiss.IndexFlatL2(dim)

# 4. Use an in-memory docstore to keep track of your documents
docstore = InMemoryDocstore()

# 5. Create the FAISS vector store manually
vectorstore = FAISS(
    embedding_function=embedding_model,       # How documents are converted to vectors
    index=index,                               # Where the vectors are stored
    docstore=docstore,                         # Keeps track of document content
    index_to_docstore_id={}                    # Mapping from FAISS index to documents
)
```
### 💡 Why Go Manual?

- ✅ **Save and load the FAISS index later**  
  Allows you to persist and reuse the vector index without rebuilding it every time.

- ✅ **Swap in custom docstores (e.g., persistent or remote ones)**  
  Useful when scaling or integrating with databases or cloud storage.

- ✅ **Full control over indexing and metadata**  
  Lets you customize how documents are tracked, mapped, or even re-ranked.

 We’re using `FAISS.from_documents()` for simplicity here,  
> but this manual route is essential as you build more advanced or production-ready RAG systems.


In [ ]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(documents, embedding_model)

### 🔍 Converting the Vector Store into a Retriever

This step turns the FAISS vector store into a retriever that can return the most relevant chunks based on a user query.

- **`search_type="similarity"`**:  
  Uses cosine similarity to find the closest matching chunks.

- **`search_kwargs={"k": 4}`**:  
  Retrieves the top 4 most relevant results for any given query.

> You can adjust `k` based on how much context you want the model to see.


In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})


### 🧠 Building the RAG Chain with GPT-4o

This block defines the brain of your RAG system — combining a powerful LLM with a custom prompt and your retriever.

- **`ChatOpenAI(model="gpt-4o")`**:  
  Uses OpenAI’s latest GPT-4o model for fast, high-quality responses.

- **`ChatPromptTemplate`**:  
  Defines the structure of the prompt. It tells the model:
  - "Here’s some context → `{context}`"
  - "Here’s the user question → `{input}`"

- **`create_stuff_documents_chain`**:  
  Feeds the retrieved chunks ("stuffed" together) and the prompt into the LLM to generate an answer.

- **`create_retrieval_chain`**:  
  Links everything: Retriever → Prompt → LLM → Response.

>| Component                  | Role                                             |
| -------------------------- | ------------------------------------------------ |
| `retriever`                | Finds the relevant documents for your query      |
| `qa_chain`                 | Stuffs docs + query into a prompt, calls the LLM |
| `create_retrieval_chain()` | Connects the two into a single callable pipeline |


> Together, this creates a full Retrieval-Augmented Generation (RAG) pipeline.

### 📦 Real-World Analogy

Imagine you're at a library asking the librarian:

> ❓ “What company does Sheryl Baxter work for?”

Here’s what happens:

- **Retriever** 🧠  
  The librarian goes and pulls out the **4 most relevant books or pages** that mention "Sheryl Baxter".

- **Stuff Chain** 📄📎  
  The librarian **copies those pages**, **staples them together**, and puts a sticky note on top that says:  
  > "Here’s the context. Now answer this question: What company does she work for?"

- **LLM** ✍️  
  The assistant reads this **whole stapled packet** and writes a **short, accurate answer** for you.

---

You never see the raw pages — just the answer.  
But the assistant’s response is powered by those retrieved documents.



In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

llm = ChatOpenAI(model="gpt-4o")   #model we use for generating the response to the user query

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the following context to answer the question briefly.\n\n{context}"),
    ("human", "{input}")
])

qa_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, qa_chain)


### ❓ Asking a Question (a.k.a. RAG in Action)

This is where it all comes together — we pass a natural language query to the `rag_chain`, and it returns an answer grounded in the CSV data.

- **`query = "..."`**  
  Define your question — it could be about any detail present in the CSV file.

- **`rag_chain.invoke({"input": query})`**  
  Pass the query to the full RAG pipeline. It:
  1. Retrieves the most relevant rows (via the FAISS index),
  2. Feeds them into the LLM using your custom prompt,
  3. Returns a concise, context-aware answer.

- **`print(response["answer"])`**  
  Displays the final response from the model.

> That’s it — ask questions and get answers directly from your structured CSV data!


In [ ]:
query = "Which company does Sheryl Baxter work for?"
response = rag_chain.invoke({"input": query})
print(response["answer"])


---

## 📘 Summary & Credits

This notebook is based on the excellent open-source repository [RAG_Techniques by NirDiamant](https://github.com/NirDiamant/RAG_Techniques).  
I referred to that work to understand how the pipeline is structured and then reimplemented the same concept in a **fully self-contained** way, but using recent models — as part of my personal learning journey.

The purpose of this notebook is purely **educational**:  
- To deepen my understanding of Retrieval-Augmented Generation systems  
- To keep a clean, trackable log of what I’ve built and learned  
- And to serve as a future reference for myself or others starting from scratch

To support that, I’ve added clear, concise markdowns throughout the notebook — explaining *why* each package was installed, *why* each line of code exists, and *how* each component fits into the overall RAG pipeline. It’s designed to help anyone (including my future self) grasp the **how** and the **why**, not just the **what**.


## 🧩 Why Start With CSV?

Many RAG tutorials use PDFs or scraped websites — but real-world customer data often lives in spreadsheets. This notebook focuses on:
- Loading CSVs as document chunks  
- Converting them to embeddings  
- Creating a retriever from FAISS  
- And answering natural language queries directly from the data  

It keeps things simple — no custom chunking (Even if we don’t manually define a splitter, the .load_and_split() method does internally chunk the CSV rows into document-like text chunks using the **RecursiveCharacterTextSplitter**.), no evaluation, no re-ranking — just **clean retrieval + grounded generation**.

## 🔁 What’s New Here?

Compared to the previous PDF-based RAG system, this version:
- Uses the latest `text-embedding-3-small` model from OpenAI  
- Demonstrates how to swap in manual FAISS indexing (for full control)  
- Highlights the use of `CSVLoader` from LangChain to semantically parse structured data  
- Adds prompt customization and GPT-4o as the default LLM  
- Keeps the notebook fully Colab-compatible and self-contained


## 💡 Final Word

This notebook is part of my larger personal project: **RAG100x** — a challenge to build and log my journney in RAG from 0 100 in the coming months.

It’s not built to impress — it’s built to **progress**.  
Everything here is structured to enable **daily iteration**, focused experimentation, and clean documentation.

If you're exploring RAG from first principles, feel free to use this as a scaffold for your own builds. And of course — check out the original repository for broader implementations and ideas.
